In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 22 19:04:59 2018

@author: amywar
"""

from graphics import *
from numpy import pi
import numpy as np
import csv

In [2]:
def movePol(obj,center,radius,angle):
    # Clone and move an object using polar coordinates
    # around a center Point object
    x0 = center.getX()
    y0 = center.getY()
    
    try:
        x1 = obj.getX()
        y1 = obj.getY()
    except AttributeError:
        try:
            objCenter = obj.getCenter()
            x1 = objCenter.getX()
            y1 = objCenter.getY()
        except AttributeError:
            print "Bad object type"
            return
    
    theta = np.arctan2(y1 - y0, x1 - x0)
    phi = theta + angle
    R = radius + np.sqrt((x1 - x0)**2 + (y1 - y0)**2)
    
    x2 = x0 + R * np.cos(phi)
    y2 = y0 + R * np.sin(phi)
    
    dx = x2 - x1
    dy = y2 - y1
    obj2 = obj.clone()
    obj2.move(dx,dy)
    return obj2

In [9]:
size = 500
numTeams = 5
colors = ['red', 'black', 'blue', 'white', 'orange']
centp = Point(size/2,size/2)
teamPts = []
teamCirc = []
goalCirc = []

In [4]:
points = np.zeros(numTeams)
gd = np.zeros(numTeams)
gf = np.zeros(numTeams)

with open('results.csv') as csvfile:
    reader = csv.reader(csvfile)
    header = reader.next()
    for row in reader:
        teamA = int(row[0]) - 1
        teamB = int(row[1]) - 1
        scoreA = int(row[2])
        scoreB = int(row[3])
        
        gd[teamA] = gd[teamA] + scoreA - scoreB
        gd[teamB] = gd[teamB] + scoreB - scoreA
        gf[teamA] = gf[teamA] + scoreA
        gf[teamB] = gf[teamB] + scoreB
        
        if scoreA > scoreB:
            points[teamA] = points[teamA] + 3
        elif scoreB > scoreA:
            points[teamB] = points[teamB] + 3
        else:
            points[teamA] = points[teamA] + 1
            points[teamB] = points[teamB] + 1

In [5]:
rank = np.zeros(numTeams)
for team in range(numTeams):
    teamPoints = points[team]
    above = len(points[points > teamPoints])
    below = len(points[points < teamPoints])
    
    rank[team] = above + 1
    
    if (above + below) != (numTeams - 1):
        teamGD = gd[team]
        tieGD = gd[points == teamPoints]
        tieGF = gf[points == teamPoints]
        above = len(tieGD[tieGD > teamGD])
        below = len(tieGD[tieGD < teamGD])
        
        rank[team] = rank[team] + above
        
        if (above + below) != (len(tieGD) - 1):
            teamGF = gf[team]
            tieGF = tieGF[tieGD == teamGD]
            above = len(tieGF[tieGF > teamGF])
            below = len(tieGF[tieGF < teamGF])
            
            rank[team] = rank[team] + above
            
            if (above + below) != (len(tieGF) - 1):
                print 'Tiebreaking unsuccessful'

In [13]:
for i in range(numTeams):
    angle = -0.5 + 2 / float(numTeams) * i
    teamPts.append(movePol(centp,centp,0.35*size,angle*pi))
    teamCirc.append(Circle(teamPts[i],size*0.075))
    teamCirc[i].setFill(colors[i])

liness = [[Line(teamPts[i],teamPts[j]) for j in range(i+1,numTeams)]
    for i in range(numTeams - 1)]

for lines in liness:
    for line in lines:
        line.setWidth(5)
        line.setFill('grey')


In [14]:
win = GraphWin('Standings',size,size)

for lines in liness:
    for line in lines:
        line.draw(win)
for circle in teamCirc:
    circle.draw(win)

In [15]:
win.close()

In [ ]:
liness